In [ ]:
import json
import numpy as np
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.prompts import PromptTemplate

# ✅ [변경 1] llama_cpp 대신 langchain_ollama 사용
from langchain_ollama import ChatOllama

# =============================================================================
# [설정] 
# =============================================================================
# 경로 대신 '모델 이름'만 알면 됩니다.
MODEL_NAME = "qwen2.5-coder:3b" 

TARGET_SCHEMA = [
    "user_id", "user_name", "phone_number", "email_address", 
    "signup_date", "last_login", "is_active", "shipping_address"
]

# =============================================================================
# [초기화] 모델 연결
# =============================================================================
print("⏳ Loading Models...")
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
target_embeddings = embed_model.encode(TARGET_SCHEMA)

# ✅ [변경 2] Ollama 객체 생성 (매우 심플함)
# format="json" 옵션을 켜면 모델이 무조건 JSON만 뱉으려고 노력합니다. (강력 추천)
llm = ChatOllama(
    model=MODEL_NAME,
    temperature=0,      # 사실 기반이므로 0 추천
    format="json"       # ✨ Ollama의 킬러 기능 (JSON 강제화)
)

# =============================================================================
# [프롬프트]
# =============================================================================
mapping_template_str = """
You are a Data Engineer. Select the best Target Column from Candidates.
[Input]
- Source: "{source_col}"
- Sample: "{sample_value}"
- Candidates: {candidates}

Analyze and select ONE target column.
Return JSON ONLY.

Example:
{{ "selected_column": "col_name", "reason": "why" }}
"""

MAPPING_PROMPT = PromptTemplate(
    template=mapping_template_str,
    input_variables=["source_col", "sample_value", "candidates"]
)

# =============================================================================
# [Node Logic]
# =============================================================================
class MappingState(TypedDict):
    source_col: str
    sample_value: str
    candidates: List[str]
    final_mapping: str
    reasoning: str

def retriever_node(state: MappingState):
    # (이전과 동일하여 생략)
    source_vec = embed_model.encode([state["source_col"]])
    similarities = cosine_similarity(source_vec, target_embeddings)[0]
    top_k_indices = np.argsort(similarities)[-3:][::-1]
    candidates = [TARGET_SCHEMA[i] for i in top_k_indices]
    return {"candidates": candidates}

def slm_reasoning_node(state: MappingState):
    print(f"🧠 [SLM] Ollama 추론 시작 ({MODEL_NAME})...")

    # 체인 연결 (LCEL 문법 사용 가능)
    # chain = Prompt | LLM
    chain = MAPPING_PROMPT | llm

    try:
        # ✅ [변경 3] 호출 방식 (invoke)
        # ChatOllama는 결과로 AIMessage 객체를 줍니다. .content로 내용을 꺼냅니다.
        response_msg = chain.invoke({
            "source_col": state["source_col"],
            "sample_value": state["sample_value"],
            "candidates": state["candidates"]
        })
        
        response_text = response_msg.content
        
        # Ollama가 format="json"이면 대부분 깔끔한 JSON이 나옵니다.
        result_json = json.loads(response_text)
        final_col = result_json.get("selected_column", "Unknown")
        reason = result_json.get("reason", "No reason")

    except Exception as e:
        print(f"❌ Error: {e}")
        final_col = "Error"
        reason = str(e)

    return {"final_mapping": final_col, "reasoning": reason}

# =============================================================================
# [Main 실행]
# =============================================================================
# (그래프 정의 및 실행 코드는 이전과 동일)
workflow = StateGraph(MappingState)
workflow.add_node("retriever", retriever_node)
workflow.add_node("reasoner", slm_reasoning_node)
workflow.set_entry_point("retriever")
workflow.add_edge("retriever", "reasoner")
workflow.add_edge("reasoner", END)
app = workflow.compile()

if __name__ == "__main__":
    result = app.invoke({
        "source_col": "m_hp", 
        "sample_value": "010-1234-5678"
    })
    print(f"\n✅ Result: {result['final_mapping']} ({result['reasoning']})")